# Minimally invasive implant of a high-density and scalable cortical microelectrode array allows multimodal neural decoding

This notebook includes functions that demostrate the following:
1. The loading of Intan-style data
2. Filtering for EEG bands
3. Triggered averaging
4. Correlation between electrode pairs
5. Hjorth activity

This notebook is intended to demonstrate methods generate the analysis for animal data presented in the Paper.

# Loading data

Data is recording in Intan-format. Please refer to the Python RHD file reader instructions and references for data loading, which can be found here: https://www.intantech.com/downloads.html?tabSelect=Software&yPos=100

In [ ]:
from load_intan_rhd_format import read_data

# define your data folder
# data_folder = '....rhd'

# Load up neural data
data = intan.read_data(data_folder)
neural_data = data['amplifier_data']

In [ ]:
import numpy as np

# Find rising edges corresponding to triggers for triggered averaging
def detect_rising_edge(sequence, threshold, cooldown, mode='index_output'):
    step_indices = []
    in_cooldown = 0
    nSize = len(sequence)
    seq_diff = np.diff(sequence)

    thres_times = np.argwhere(seq_diff > threshold)[:,0]
    step_indices.append(thres_times[0])
    for t in thres_times[1:]:
        if t - step_indices[-1] > cooldown:
            step_indices.append(t)

    return np.array(step_indices)

trig = detect_rising_edge(data['board_dig_in_data'][0,:].astype(int), threshold=0.5, cooldown=500, mode='index_output')

# Compute EEG bands

In [ ]:
from scipy import signal

# Define functions for filters

def butter_lowpass_filter(data, cutoff, fs, order):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    sos = signal.butter(order, normal_cutoff, btype='low', analog=False, output='sos')
    y = signal.sosfiltfilt(sos, data)
    return y

def butter_bandpass_filter(data, cutoff, fs, order):
    nyq = 0.5 * fs
    normal_cutoff = np.array(cutoff) / nyq
    # Get the filter coefficients 
    sos = signal.butter(order, normal_cutoff, btype='bandpass', analog=False, output='sos')
    y = signal.sosfiltfilt(sos, data)
    return y

In [ ]:
# Example of computing an EEG band
neural_bp = butter_bandpass_filter(neural_data, [4,300], fs=20000, order=5)

# Compute triggered averaging

In [ ]:
data_window = np.array((0,50)).astype(int)
data_window *= 20
amp_trial = np.zeros((100, 1024, int((data_window[1]-data_window[0])))) # computing for 100 trials
count = 0 # Using count instead of enumerate here, in case you want to add critera for rejecting individual trials 
for t in trig[:100]:
    # Apply common mode subtraction
    amp_trial[count, selected_chs, :] = neural_bp[selected_chs,t+data_window[0]:t+data_window[1]] - np.mean(neural_bp[selected_chs,t+data_window[0]:t+data_window[1]], axis=0, keepdims=True)
    count += 1
ssep = np.mean(amp_trial,axis=0)

# Correlation between electrode pairs

In [ ]:
Fs = 20000 # sampling frequency
neural_data_filt = butter_bandpass_filter(neural_data, cutoff=[0.1, 300], fs=20000, order=5)

corr2_sum = np.zeros((1024,1024))
corr_avg = np.zeros((1024,1024))
count = 0

# Compute the correlation coefficients and R^2
for t in trig:
    segment = neural_data_filt[:,t:t+Fs*2]
    corr = np.corrcoef(segment)
    corr_avg += corr
    corr2_sum += corr**2
    count += 1
corr_avg /= count
corr2_sum /= count

# Hjorth parameters

In [ ]:
Fs = 20000 # sampling frequency
neural_data_filt = butter_bandpass_filter(neural_data, cutoff=[0.1, 300], fs=20000, order=5)

hjorth = np.zeros((40, 1024))
count = 0

# Compute the correlation coefficients and R^2
for k, t in enumerate(trig):
    segment = neural_data_filt[:,t:t+Fs*2]
    hjorth[k, :] = np.var(segment, axis=-1)
    count += 1
hjorth /= count